# Deploy a LLM

Before proceeding with our lab, we will deploying a large language model from the Jumpstart model hub to a real-time SageMaker Endpoint.

The model we will pick is the Finnish language fine-tuned model created by TurkuNLP research group.


In [ ]:
%%capture
# Upgrade SageMaker SDK and get Widgets
!pip install ipywidgets==7.7 --quiet
!pip install --upgrade sagemaker --quiet

In [ ]:
import ipywidgets as widgets
import json
from sagemaker import Session
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models
from sagemaker.jumpstart.model import JumpStartModel
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

In [ ]:
# Let's get all Extractive Question Answer models from Jumpstart
all_models = list_jumpstart_models(filter='task == eqa')

# display the model-ids in a dropdown to select a model for inference.
model_dropdown = widgets.Dropdown(
    options=all_models,
    description="Select a model",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

In [ ]:
display(model_dropdown)

In [ ]:
model_id = model_dropdown.value
my_model = JumpStartModel(model_id=model_id)
my_model.__dict__

Notice that for the Jumpstart models, we have the S3 asset, inference scripts, container definitions and so on.

# Let's Deploy the TurkuNLP LLM

While not immediately available in JumpStart, various models in different sizes trained by TurkuNLP can be found here in Hugging Face: https://huggingface.co/TurkuNLP. The Bloom model is large, and since we are pulling these models to local disk we will need to go with a more reasonable option such as https://huggingface.co/TurkuNLP/gpt3-finnish-large .

Note from the page that these *models are pure language models, meaning that they are not instruction finetuned for dialogue or answering questions.*

In [ ]:
role = Session().get_caller_identity_arn()

MODEL_ID = 'TurkuNLP/gpt3-finnish-large'
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID': MODEL_ID,
	'SM_NUM_GPUS': json.dumps(1)
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.8.2"),
	env=hub,
	role=role, 
)

huggingface_model.__dict__

In [ ]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    endpoint_name=MODEL_ID.replace('/','-')
	initial_instance_count=1,
	instance_type="ml.g5.xlarge",
	container_startup_health_check_timeout=300,
)

In [ ]:

prompt = """Onko seuraava lause positiivinen vai negatiivinen?
Lause: Täällä on erittäin hauskaa!
Vastaus: positiivinen

Onko seuraava lause positiivinen vai negatiivinen?
Lause: elokuva oli upea!
Vastaus:"""

parameters = {
        "top_k": 6,
        "top_p": 0.5,
        "max_new_tokens": 30,
        "temperature": 0.8,
        "no_repeat_ngram_size": 2
}

# send request to model
print(predictor.predict({
	"inputs": prompt,
    "parameters" : parameters
})[0]['generated_text'])